In [1]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


/home/brilworks/Desktop/Learning/model_finetuning/vevnv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


GPU Available: False
GPU Name: No GPU


In [27]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


ValueError: Could not load model meta-llama/Llama-3.2-3B-Instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/home/brilworks/Desktop/Learning/model_finetuning/vevnv/lib/python3.12/site-packages/transformers/pipelines/base.py", line 292, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/brilworks/Desktop/Learning/model_finetuning/vevnv/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py", line 571, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/brilworks/Desktop/Learning/model_finetuning/vevnv/lib/python3.12/site-packages/transformers/mode

In [ ]:
import numpy
print(numpy.__version__)


2.3.0


In [ ]:
from ollama import chat

stream = chat(
    model='llama3.2:1b',
    messages=[{'role': 'user', 'content': 'how are you?'}],
    stream=True
)

for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)


I'm doing well, thank you for asking. Is there something I can help you with or would you like to talk about yourself?

In [19]:
!ollama create hasmukh_mer-llama3.2:1b -f Modelfile



Error: (line 4): command must be one of "from", "license", "template", "system", "adapter", "parameter", or "message"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from ollama import chat

response = chat(model='hasmukh-mer-llama3.2:1b', messages=[
    {'role': 'user', 'content': 'Who is Hasmukh Mer?'}
])

print(response['message']['content'])


I don't have any information on a person named Hasmukh Mer. Could you please provide more context or clarify who Hasmukh Mer is? I'll do my best to help.


In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./finetuned_cpu")
tokenizer = AutoTokenizer.from_pretrained("./finetuned_cpu")

prompt = "Who is hasmukh mer"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



ValueError: Unrecognized model in ./finetuned_cpu. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v3, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, ele